## Коллокации

Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе. Также часто значения коллокации не могут быть выведены лишь из значений, входящих в них слов.

Нахождение коллокаций на практике - это какая-то фильтрация нграммов в корпусе. Прежде чем, переходить к этому, вспомним как в питоне можно вытащить нграммы. Заодно повторим и нормализацию.

Возьмем какой-нибудь кусочек текста для примера.

In [191]:
text = """Доклад Андрея можно использовать как справочник и how-to по отладке проблем с утечкой нативной памяти на примере non-heap памяти. 

Доклад полезен для понимания, кто и что съедает память. Андрей показывает инструменты для анализа памяти, в том числе AsyncProfiler, который встроен в Idea: «The upcoming IntelliJ IDEA 2018.3 integrates a low overhead sampling profiler that can profile JVM and Native code – Async profiler».

Мы рекомендуем всем посмотреть его и прогнать по тем же шагам свои модули и микросервисы."""

### Задание. 

Напишите функцию, которая будет принимать текст, разбивать его на токены и группировать токены в нграммы. Параметр N - должен быть задаваемым.

In [83]:
import itertools
from pymorphy2 import MorphAnalyzer
from string import punctuation
morph = MorphAnalyzer()

def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


def ngrammer(tokens, n=2):
    ## ваш код здесь
    
    return ngrams
    

In [193]:
# работать должно вот так
ngrammer(normalize(text))[:10]

[('доклад', 'андрей'),
 ('андрей', 'можно'),
 ('можно', 'использовать'),
 ('использовать', 'как'),
 ('как', 'справочник'),
 ('справочник', 'и'),
 ('и', 'how-to'),
 ('how-to', 'по'),
 ('по', 'отладка'),
 ('отладка', 'проблема')]

Теперь перейдем к самим коллокациям.

## Данные 

Возьмём небольшую коллекцию научных статей с киберленинки. Они переведены в текст с помощью pdf2text, поэтому там полно всякого мусора.

In [5]:
import pandas as pd
import os
from collections import Counter, defaultdict
import numpy as np

In [86]:
data_files = ['../data/'+file for file in os.listdir('../data')  if 'ng' in file and file.endswith('.jsonlines')]

In [87]:
data_files

['../data/ng_0.jsonlines']

In [88]:
data = pd.concat([pd.read_json(file, lines=True) for file in data_files[:2]], ignore_index=True)

In [89]:
pd.set_option('display.max_colwidth', 1000)

In [194]:
data.head(1)

,content,keywords,summary,title,url
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html


Преобразуем тексты в токены с помощью функции, которую написали.

In [92]:
texts = data['content'].apply(normalize).tolist()

In [93]:
ngrammer(texts[1])[:20]

[('вчера', '«газпром»'),
 ('«газпром»', 'снизить'),
 ('снизить', 'верхний'),
 ('верхний', 'планка'),
 ('планка', 'прогноз'),
 ('прогноз', 'собственный'),
 ('собственный', 'добыча'),
 ('добыча', 'газа'),
 ('газа', 'в'),
 ('в', '2020'),
 ('2020', 'год'),
 ('год', 'через'),
 ('через', '12'),
 ('12', 'год'),
 ('год', 'концерн'),
 ('концерн', 'собираться'),
 ('собираться', 'добывать'),
 ('добывать', 'около'),
 ('около', '620–640'),
 ('620–640', 'миллиард')]

Самый простой способ найти устойчивые сочетания - просто посчитать всё и взять самые частотные.

In [94]:
word_counter = Counter()

for text in texts:
    word_counter.update(ngrammer(text, 4))


Посмотрим, что получилось.

In [96]:
word_counter.most_common(15)

[(('в', 'тот', 'число', 'и'), 100),
 (('в', 'то', 'же', 'время'), 100),
 (('дело', 'в', 'тот', 'что'), 51),
 (('несмотря', 'на', 'то', 'что'), 48),
 (('в', 'связь', 'с', 'это'), 45),
 (('в', 'то', 'время', 'как'), 40),
 (('миллиард', 'куб', 'метр', 'газа'), 40),
 (('до', 'сей', 'пора', 'не'), 36),
 (('говорить', 'о', 'тот', 'что'), 29),
 (('абхазия', 'и', 'южный', 'осетия'), 28),
 (('президент', 'рф', 'владимир', 'путин'), 27),
 (('в', 'тот', 'число', 'в'), 27),
 (('не', 'говорить', 'уже', 'о'), 24),
 (('состоять', 'в', 'тот', 'что'), 24),
 (('заключаться', 'в', 'тот', 'что'), 23)]

В списке много всяких чисел, однобуквеных слов и стоп-слов. 

Добавим какие-нибудь ограничения к коду выше, чтобы биграммы получались почище.

In [97]:
from nltk.corpus import stopwords

In [98]:
stops = set(stopwords.words('russian'))

In [99]:
def ngrammer(tokens, n=2, stops=set()):
    
#     tokens = normalize(text)
    tokens = [token for token in tokens if token not in stops]
    ngrams = list(zip(*(itertools.islice(tokens, i, None) for i in range(n))))
    return ngrams

In [102]:
word_counter = Counter()

for text in texts:
    word_counter.update(ngrammer(text, n=2, stops=stops))


In [103]:
word_counter.most_common(15)

[(('–', 'это'), 778),
 (('2017', 'год'), 328),
 (('2016', 'год'), 274),
 (('–', 'сказать'), 221),
 (('владимир', 'путин'), 217),
 (('прошлое', 'год'), 211),
 (('миллиард', 'доллар'), 195),
 (('куб', 'метр'), 183),
 (('точка', 'зрение'), 178),
 (('это', 'год'), 171),
 (('год', 'назад'), 166),
 (('сей', 'пора'), 166),
 (('речь', 'идти'), 161),
 (('2015', 'год'), 151),
 (('весь', 'это'), 150)]

В списке есть коллокации, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности. 

In [104]:
def scorer_simple(word_count_a, word_count_b, bigram_count, _):
    try:
        score = bigram_count/((word_count_a+word_count_b)-bigram_count)
    
    except ZeroDivisionError:
        return 0
    
    return score

Сделаем функцию, которая будет делать счетчики для слов и биграммов.

In [129]:
def collect_stats(texts):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    word_counter = Counter()
    bigram_counter = Counter()
    
    for text in texts:
        word_counter.update(text)
        bigram_counter.update(ngrammer(text, 2, stops))
    
    return word_counter, bigram_counter

И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрку.

In [130]:
def score_bigrams(word_counter, bigram_counter, scorer, threshold=-100000):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(word_counter)
    for bigram in bigram_counter:
        score = scorer(word_counter[bigram[0]], word_counter[bigram[1]], 
                       bigram_counter[bigram], len_vocab)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [131]:
word_counter, bigram_counter = collect_stats(texts)

In [132]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_simple)

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [133]:
bigram2score.most_common(15)

[(('н036', '«белка»'), 1.0),
 (('«three', 'zero»'), 1.0),
 (('цесаревич', 'цесаревна'), 1.0),
 (('метареализм', 'семиотика'), 1.0),
 (('rahmstorf', 'потсдамский'), 1.0),
 (('замесить', 'пресмыкание'), 1.0),
 (('виснуть', 'парусина'), 1.0),
 (('near', 'infrared'), 1.0),
 (('«империалистический', 'происки»'), 1.0),
 (('свирск', 'черемхово'), 1.0),
 (('золя', 'эдмона'), 1.0),
 (('розвальни', 'кочка'), 1.0),
 (('эльдар', 'рязанов'), 1.0),
 (('gaffney', 'cline&associates'), 1.0),
 (('water', 'congress'), 1.0)]

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграмма.

In [135]:
def scorer(word_count_a, word_count_b, bigram_count, _, minimum_count=10):
    try:
        score = ((bigram_count - minimum_count) / ((word_count_a + word_count_b) - bigram_count))
    except ZeroDivisionError:
        return 0
    
    return score

In [136]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer)

In [137]:
bigram2score.most_common(15)

[(('саудовский', 'аравия'), 0.7111111111111111),
 (('red', 'bull'), 0.7027027027027027),
 (('бенедикт', 'xvi'), 0.6285714285714286),
 (('сей', 'пора'), 0.5591397849462365),
 (('точка', 'зрение'), 0.5419354838709678),
 (('toro', 'roso'), 0.5),
 (('чеченец', 'ингуш'), 0.41379310344827586),
 (('искусственный', 'интеллект'), 0.40869565217391307),
 (('homo', 'sapiens'), 0.3888888888888889),
 (('куб', 'метр'), 0.3581780538302277),
 (('кольцевой', 'автогонки'), 0.35),
 (('new', 'york'), 0.3333333333333333),
 (('гарф.ф', '6991'), 0.3125),
 (('нико', 'росберг'), 0.3108108108108108),
 (('washington', 'post'), 0.30434782608695654)]

В статье про Word2Vec для создания нграммов использовалась такая функция:

In [138]:
def scorer(word_count_a, word_count_b, bigram_count,len_vocab, minimum_count=10):

    try:
        score = ((bigram_count - minimum_count) / (word_count_a * word_count_b)) * len_vocab
    except ZeroDivisionError:
        return 0
    
    return score

Посмотрим, отличается ли она от нашей.

In [139]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer)

In [140]:
bigram2score.most_common(15)

[(('toro', 'roso'), 1302.025),
 (('homo', 'sapiens'), 1191.3954248366013),
 (('new', 'york'), 1157.3555555555556),
 (('гарф.ф', '6991'), 1085.0208333333333),
 (('кольцевой', 'автогонки'), 1065.985380116959),
 (('бенедикт', 'xvi'), 1021.1960784313725),
 (('red', 'bull'), 1016.5960960960962),
 (('чеченец', 'ингуш'), 979.5799373040753),
 (('washington', 'post'), 932.3964194373401),
 (('«войско', 'оон»'), 797.1581632653061),
 (('dragon', 'eye'), 759.5145833333333),
 (('вальттери', 'боттас'), 754.7971014492754),
 (('умберто', 'эко'), 649.8520499108735),
 (('тереза', 'мэй'), 642.9753086419753),
 (('ренэ', 'герр'), 642.9753086419753)]

Популярная метрика для коллокация - PMI. Она очень похожа на то, что мы уже написали.

Только вместо количества вохождений используют частотность.

In [141]:
def scorer_pmi(word_count_a, word_count_b, bigram_count, _, corpus_size, minimum_count=5):
    score = ((bigram_count/corpus_size) / ((word_count_a/corpus_size) * (word_count_b/corpus_size)))
    
    return np.log(score) / -np.log((bigram_count/corpus_size))

def score_bigrams(word_counter, bigram_counter, scorer, threshold=-100000):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(word_counter)
    corpus_size = sum(word_counter.values())
    
    for bigram in bigram_counter:
        score = scorer(word_counter[bigram[0]], word_counter[bigram[1]], 
                       bigram_counter[bigram], len_vocab, corpus_size)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [142]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_pmi)

In [143]:
bigram2score.most_common(15)

[(('н036', '«белка»'), 1.0),
 (('«three', 'zero»'), 1.0),
 (('цесаревич', 'цесаревна'), 1.0),
 (('метареализм', 'семиотика'), 1.0),
 (('rahmstorf', 'потсдамский'), 1.0),
 (('замесить', 'пресмыкание'), 1.0),
 (('виснуть', 'парусина'), 1.0),
 (('near', 'infrared'), 1.0),
 (('«империалистический', 'происки»'), 1.0),
 (('свирск', 'черемхово'), 1.0),
 (('золя', 'эдмона'), 1.0),
 (('розвальни', 'кочка'), 1.0),
 (('эльдар', 'рязанов'), 1.0),
 (('gaffney', 'cline&associates'), 1.0),
 (('water', 'congress'), 1.0)]

Ещё для оценки нграммов используют статистические тесты. Например, ttest выражается в частотах слов вот так:

In [144]:
def scorer_ttest(word_count_a, word_count_b, bigram_count, _, corpus_size, minimum_count=5):
    mu = ((word_count_a/corpus_size) * (word_count_b/corpus_size))
    x_ = (bigram_count/corpus_size)
    score = (x_ - mu) / np.sqrt(x_/corpus_size)
    
    return score

In [145]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_ttest)

In [188]:
bigram2score.most_common(15)

[(('–', 'это'), 24.609137170357467),
 (('2017', 'год'), 17.970580269597036),
 (('2016', 'год'), 16.436696795603797),
 (('владимир', 'путин'), 14.710647284146532),
 (('прошлое', 'год'), 14.366229586134002),
 (('–', 'сказать'), 13.961663861479058),
 (('миллиард', 'доллар'), 13.941389113195168),
 (('куб', 'метр'), 13.519292320540593),
 (('точка', 'зрение'), 13.33648192767963),
 (('сей', 'пора'), 12.879710463131945),
 (('год', 'назад'), 12.761195104210106),
 (('речь', 'идти'), 12.668439331475213),
 (('2015', 'год'), 12.201752933666516),
 (('2018', 'год'), 11.901448689899807),
 (('2014', 'год'), 11.708786199682178)]

Во всех случаях выше мы считали нграммами только слова, которые встречаются друг за другом. Но в нграммы часто можно ещё что-то вставить. Например, "принять участие" может превратиться в "принять самое активное/непосредственное участие". 

Чтобы отловить такие случаи можно считать нграммами слова, которые встречаются внутри какого-то окна. И считать по ним все те же метрики.

Можно ещё посчитать стандартное отклонение расстояния между двумя словами. Если оно маленькое - слова обычно стоят на строгой позиции по отношению друг к другу.

In [149]:
def get_window_stats(texts, window=8):
    
    bigrams = defaultdict(list)
    
    # проходим окном по текстам 
    # берем первое слово и считаем его целевым
    # проходим по остальным словам и их индексам
    # добавляем в словарь пары (целевое слов, текущее слово)
    # и добавляем индекс текущего в список этой пары
    # так мы получаем (слово_1,слово_2):[1,2,1,1,3,2]
    # порядок в этом случае учитывается - (слово_2, слово_1) - другая запись
    for text in texts:
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[1:]:
                bigrams[(target, word)].append(j)
    
    bigrams_stds = Counter()
    for bigram in bigrams:
        # выкидываем биграмы встретившиеся < 5 раз
        if len(bigrams[bigram]) > 5:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    
    return bigrams_stds

In [150]:
bigram2std = get_window_stats(texts)

In [152]:
bigram2std.most_common()[:-10:-1]

[(('театр', 'твд'), 0.0),
 (('избавиться', 'от'), 0.0),
 (('власть', 'великобритания'), 0.0),
 (('en', 'group'), 0.0),
 (('много', 'ни'), 0.0),
 (('бутерброд', 'с'), 0.0),
 (('предвыборный', 'гонка'), 0.0),
 (('дмитрий', 'песок'), 0.0),
 (('бурят-монгольский', 'поэзия'), 0.0)]

Можно применять расширить размер нграмма, а можно последовательно преобразовывать один и тот же текст, на каждом шагу собирая новые биграммы.

Напишием такую функцию.

In [153]:
def bigram_text(text, bigram2score):
    new_text = []
    i = 0
    
    while i < (len(text)-1):
        bigram = (text[i], text[i+1])
        if bigram in bigram2score:
            new_text.append('_'.join(bigram))
            i += 2
        else:
            new_text.append(text[i])
            i += 1
    else:
        if i == (len(text)-1):
            new_text.append(text[i])
    
    return new_text

In [159]:
bigram2score = score_bigrams(word_counter, bigram_counter, scorer_ttest)

In [160]:
bi_texts = [bigram_text(texts[i], bigram2score) for i in range(len(texts))]

In [161]:
bigram_count, trigram_count = collect_stats(bi_texts)

In [162]:
trigram2score = score_bigrams(bigram_count, trigram_count, scorer_ttest)

In [163]:
tri_texts = [bigram_text(bi_texts[i], trigram2score) for i in range(len(bi_texts))]

In [165]:
tri_texts[1][:10]

['вчера_«газпром»_снизить_верхний',
 'планка_прогноз_собственный_добыча',
 'газа',
 'в',
 '2020_год',
 'через',
 '12_год_концерн_собираться',
 'добывать_около_620–640_миллиард',
 'куб_метр',
 'в']

Сюда тоже можно добавить окно и использовать метрику со стандартным отклонением посчтитанную выше. 

По этой ссылке можно прочитать про другие метрики.

http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

### Все готовое

Писать все это самому конечно не обязательно.

Удобно пользоваться phraser из gensim'а. Он собирает статистику по корпусу, а затем склеивает слова в биграммы. Так как мы сделали выше. 

In [171]:
import gensim

In [172]:
# собираем статистики
ph = gensim.models.Phrases(texts)

In [173]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

По умолчанию там используется метрики из статьи про ворд2век и ещё есть нормализованные pmi.
Если не нравятся функции оценки, то ему можно подать любую другую функцию. Интерфейс у функции там почти точно такой же как и у наших.

In [137]:
?gensim.models.Phrases

In [186]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[texts])
p2 = gensim.models.phrases.Phraser(ph2)

In [187]:
p2[p[texts[0]]][:20]

['многие',
 'интересоваться',
 'зачем',
 'нужный',
 '«яблоку»',
 'молодёжный',
 'фракция',
 'основной_задача',
 '«молодёжный',
 '«яблока»',
 'являться',
 'привлечение',
 'молодая_человек',
 'к',
 'участие',
 'в',
 'выборы',
 'и',
 'деятельность',
 'партия']

Ну и наконец нграммы есть в нлтк. Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [177]:
import nltk
from nltk.collocations import *

In [178]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [179]:
finder2 = BigramCollocationFinder.from_documents(texts)

In [180]:
finder3 = TrigramCollocationFinder.from_documents(texts)

In [182]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

[('один', 'из'),
 ('тот', 'что'),
 ('а', 'также'),
 ('при', 'это'),
 ('2017', 'год'),
 ('не', 'только'),
 ('точка', 'зрение'),
 ('то', 'есть'),
 ('сей', 'пора'),
 ('тот', 'число'),
 ('куб', 'метр'),
 ('владимир', 'путин'),
 ('2016', 'год'),
 ('тот', 'же'),
 ('потому', 'что'),
 ('миллиард', 'доллар'),
 ('о', 'тот'),
 ('прежде', 'всего'),
 ('кроме', 'тот'),
 ('до', 'сей')]

In [183]:
finder3.nbest(trigram_measures.pmi, 20)

[('1947–2001»', 'monterey', 'ca'),
 ('50-летие', 'rolling', 'stones'),
 ('acs', 'nano', 'letters'),
 ('areva', 'edf', 'alstom'),
 ('armored', 'multi-purpose', 'vehicles'),
 ('atr', 'ленур', 'ислям'),
 ('bad', 'can', 'it'),
 ('bourgeois', '«эпатировать', 'буржуа»'),
 ('bundesanstalt', 'fuer', 'geowissenschaften'),
 ('can', 'it', 'be'),
 ('charge', 'ion', 'battery'),
 ('citizens', '1947–2001»', 'monterey'),
 ('commitment', 'competence', 'consensus'),
 ('corriere', 'della', 'sera'),
 ('della', 'sera', 'папа-на-покой'),
 ('diyanet', 'isleri', 'turk-islam'),
 ('dux', 'recording', 'producers'),
 ('edf', 'alstom', 'schneider'),
 ('egf', 'gazprom', 'monitor'),
 ('espanola', 'чть', 'прад')]